### Ejemplo de funcion Minhash

En este <b>minhash</b> de ejemplo la probabilidad depende de la distancia de la siguiente manera: <b>p = 1 - d</b>

<img src="ejemplo_minhash.png">

Para objetos a <b>distancia</b> 0.2 nuestra <b>probabilidad de colision</b> es 0.8, es decir que a partir de hashear un query podemos recuperar un 80% de los objetos que estan a distancia 0.2 o menos. El 20% de objetos que son cercanos pero no recuperamos lo denominamos <b>falsos negativos</b>. 

Para objetos a <b>distancia</b> 0.6 o mayor entonces la probabilidad de recuperarlos es 0.4. Si consideramos que a objetos a distancia 0.6 o mayor no son cercanos igual vamos a estar recuperando un 40% de ellos al acceder a nuestra tabla de hashing. Estos objetos que no son cercanos pero que igualmente colisionan los llamamos <b>falsos positivos</b>.

# Reduccion de falsos positivos

Es posible reducir la cantidad de falsos positivos usando mas de una funcion de minhash sobre la misma tabla. Si usamos <b>r</b> funciones minhash en la misma tabla cada una de ellas nos dara como resultado un bucket. Los documentos cantidatos surgen entonces de la intersecciones de los documentos que estan en los r buckets que visitaremos.

Por ejemplo con r = 3 aplicamos tres minhashes diferentes y obtenemos h1(x)=21, h2(x)=30, h3(x)=5.
- Accedemos al bucket 21 y recuperamos los objetos cuyos ids son {101,204,305,306,307,411,508}.
- Accedemos al bucket 30 y recuperamos los objetos cuyos ids son {101,208,210,305,508,611,903}.
- Accedemos al bucket 4  y recuperamos los objetos cuyos ids son {101,208,209,305}.

Realizamos la <b>interseccion</b> de ids y queda: {101,305}. Asi que estos son los candidatos a ser cercanos a nuestro objeto query.

<img src="reduccion_falsos_positivos.png">

Notar como para distancia mayor a 0.8 la probabilidad de colision es ultrabaja, por lo tanto no colisionara y no se producira un falso positivo. 

Pero notar como caen las probabilidades de colision a poca distancia. Esto aumenta los falsos negativos. Ya que aumenta la cantidad de veces que no colisiona cuando la distancia es chica, osea cuando deberia.

# Reduccion de falsos negativos

Reducir los falsos positivos implica poder recuperar mas objetos. Es decir, que no se nos escapen objetos que son similares a la query pero no son recuperados por LSH. La idea consiste en usar mas de una tabla de hashing.

Probabilidad de que dos claves colisionen: p <br>
Prababilidad de que dos claves no colisionen: 1 - p <br>
Prababilidad de que dos claves no colisionen en ninguna de las tablas: (1 - p)^b <br>
Probabilidad de que dos claves colisionen en alguna tabla: 1 - (1 - p)^b <br>

<img src="reduccion_falsos_negativos.png">

# Combinando los dos metodos

Podemos lograr un numero tan bajo como deseemos de falsos positivos y falsos negativos usando ambos metodos combinados, es decir b tablas de hashing y r minhashes sobre cada tabla. La cantidad total de minhashes es entonces b*r.

<img src="combinando.png">

<img src="combinar_ambos_metodos.png">

# Encontrando los valores de b y r (grid-search)

En general cuando usamos LSH empezamos definiendo la distancia que vamos a usar. Para cada distancia hay uno, varios o ningun minhash posible. Elegimos un minhash y obtenemos, de forma teorica o mediante una simulacion, la probabilidad de colision en funcion de la distancia.

Luego definimos que valores queremos para <b>d1</b>, <b>d2</b>, <b>p1</b> y <b>p2</b>. Por ejemplo, suponiendo que la distancia es un valor entre 0 y 1.

Tambien, suponemos que usamos un minhash tal que <b>p = 1 - d</b>, es decir que la probabilidad de colision es la inversa de la distancia, con distancias entre 0 y 1.

<hr>

<b>Definimos objetivo</b>. Encontrar valores de b y r tales que:
- <b>d = 0.2</b> | <b>p1</b> igual o mayor a <b>0.95</b>
- <b>d = 0.4</b> | <b>p2</b> igual o menor a <b>0.05</b>

In [1]:
# Funcion que calcula la probabildad de colision.
# Parametros:
#### r: Cantidad de funciones de hash.
#### b: Cantidad de tablas de hash.
#### d: Distancia entre los puntos.
def prob_colision(r,b,d):
   p = 1 - d
   return (1 - (1 - p**r)**b) 

Probamos todas las conbinaciones de r y b hasta que damos con nuestro objetivo de probabilidades.

In [2]:
def search_grid(d1,d2,p1,p2):
    for r in range(0, 300):
        for b in range(0, 300):
            if (prob_colision(r,b,d1) > p1 and prob_colision(r,b,d2) < p2):
                return 'r = ' + str(r) + ', ' +  'b = ' + str(b)

In [3]:
d1 = 0.2
d2 = 0.4
p1 = 0.95
p2 = 0.05
search_grid(d1,d2,p1,p2)

'r = 15, b = 84'

Probando con todas las conbinaciones de b y r encontramos que r = 15 y b = 84 cumple con nuestro objetivo.